In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt

In [2]:
df = gpd.read_file('election_districts.geojson')

In [3]:
df = df.rename(columns = {'elect_dist': 'KEY'})

In [4]:
df.head()

,KEY,shape_area,shape_leng,geometry
0,23002,5646613.46387,13511.9504491,(POLYGON ((-73.91313180128004 40.5646382439460...
1,23004,15164800.4804,27487.6508352,(POLYGON ((-73.86495467575234 40.5666314776016...
2,23005,6156201.86738,16269.6434127,(POLYGON ((-73.85901463392794 40.5779871113073...
3,23006,3188045.51181,8261.76374922,(POLYGON ((-73.85324184655673 40.5713836120550...
4,23007,3454288.97721,10778.549738,(POLYGON ((-73.84850444718799 40.5721484475809...


In [5]:
temp_df = pd.read_csv('voter_counts/registered_voters.csv')

In [6]:
df['KEY'] = df['KEY'].astype(int)

In [7]:
df = df.merge(right = temp_df, on = 'KEY', how = 'inner')

In [8]:
df = df.drop(['Unnamed: 0', 'ELECTION DIST', 'STATUS'], axis = 1)

In [9]:
columns_to_float = ['DEM'
       ,'REP', 'CON', 'WOR', 'GRE', 'LBT', 'IND', 'SAM', 'OTH', 'BLANK',
       'TOTAL']

for col in columns_to_float:
    df[col] = df[col].apply(lambda x: x if type(x) == float else float(str(x).replace('$', '').replace(',', '').replace(' ', '')))

In [10]:
df[['DEM'
       ,'REP', 'CON', 'WOR', 'GRE', 'LBT', 'IND', 'SAM', 'OTH', 'BLANK',
       'TOTAL']]

,DEM,REP,CON,WOR,GRE,LBT,IND,SAM,OTH,BLANK,TOTAL
0,423.0,428.0,49.0,3.0,1.0,0.0,44.0,0.0,0.0,184.0,1132.0
1,98.0,63.0,4.0,0.0,0.0,0.0,9.0,0.0,0.0,29.0,203.0
2,433.0,342.0,16.0,2.0,1.0,1.0,28.0,0.0,1.0,192.0,1016.0
3,440.0,368.0,8.0,0.0,4.0,0.0,38.0,0.0,0.0,169.0,1027.0
4,433.0,348.0,16.0,1.0,0.0,1.0,39.0,0.0,0.0,143.0,981.0
...,...,...,...,...,...,...,...,...,...,...,...
4407,139.0,8.0,1.0,0.0,1.0,0.0,8.0,0.0,0.0,29.0,186.0
4408,633.0,36.0,0.0,0.0,1.0,0.0,28.0,0.0,0.0,114.0,812.0
4409,615.0,148.0,7.0,6.0,0.0,0.0,33.0,0.0,2.0,303.0,1114.0
4410,554.0,151.0,6.0,5.0,3.0,0.0,17.0,0.0,2.0,309.0,1047.0


In [11]:
temp_df = pd.read_csv('12cdenhanced.csv', low_memory=False)

In [12]:
temp_df['KEY'] = temp_df.Precinct.apply(lambda x: int(x[-2:] + x[-6:-3]))

temp_df['EstHomeValue'] = temp_df['EstHomeValue'].apply(lambda x: x if type(x) == float else int(x.replace('$', '').replace(',', '').replace(' ', '')))
temp_df['EstimatedIncomeAmount'] = temp_df['EstimatedIncomeAmount'].apply(lambda x: x if type(x) == float else int(x.replace('$', '').replace(',', '').replace(' ', '')))

temp_df['Voters_Gender'] = temp_df['Voters_Gender'].apply(lambda x: 1 if x == 'M' else 0)
temp_df = pd.get_dummies(temp_df, columns = ['Broad Ethnic Groupings', 'Household Primary Language', 'Religions_Description'], dummy_na = True)

In [13]:
primaries = ['Presidential_Primary_1992-04-07',
       'Presidential_Primary_1996-03-07',
       'Presidential_Primary_2000-03-07',
       'Presidential_Primary_2004-03-02',
       'Presidential_Primary_2008-02-05',
       'Presidential_Primary_2012-04-24',
       'Presidential_Primary_2016-04-19']

for primary in primaries:
    temp_df[primary] = temp_df[primary].apply(lambda x: 1 if x == 'Y' else 0)

In [14]:
temp_df[['KEY', 'Voters_Age', 'EstHomeValue', 'MedianEducationYears', 'EstimatedIncomeAmount']].groupby('KEY').median()

,Voters_Age,EstHomeValue,MedianEducationYears,EstimatedIncomeAmount
KEY,,,,
30019,86.0,625000.0,12.0,64999.0
30038,43.0,624999.0,12.0,46999.0
30056,43.0,416847.5,15.0,84721.0
30064,41.5,663264.0,13.0,83729.0
30066,40.0,604166.0,13.0,47499.0
...,...,...,...,...
76099,56.5,480768.0,15.0,196000.0
76100,58.0,649999.0,15.0,149553.0
76101,71.0,625000.0,16.0,126500.0


In [15]:
df = df.merge(right = temp_df[['KEY', 'Voters_Age', 'EstHomeValue', 'MedianEducationYears', 'EstimatedIncomeAmount']].groupby('KEY').median(), on = 'KEY', how = 'inner')

In [16]:
len(df)

544

In [17]:
temp_df[['KEY', 'Voters_Gender', 'Vote_Frequency']].groupby('KEY').mean()

,Voters_Gender,Vote_Frequency
KEY,,
30019,1.000000,5.000000
30038,0.549550,3.684685
30056,0.533981,3.980583
30064,0.543478,3.652174
30066,0.416667,4.000000
...,...,...
76099,0.394737,4.122807
76100,0.382979,4.049645
76101,0.323944,4.028169


In [18]:
df = df.merge(right = temp_df[['KEY', 'Voters_Gender', 'Vote_Frequency']].groupby('KEY').mean(), on = 'KEY', how = 'inner')

In [19]:
temp_df[['KEY', 'Broad Ethnic Groupings_East and South Asian',
       'Broad Ethnic Groupings_European',
       'Broad Ethnic Groupings_Hispanic and Portuguese',
       'Broad Ethnic Groupings_Likely African-American',
       'Broad Ethnic Groupings_Other', 'Broad Ethnic Groupings_nan',
       'Household Primary Language_Assimilated - English Speaking',
       'Household Primary Language_Bilingual - English Primary',
       'Household Primary Language_Bilingual - Native Language Primary',
       'Household Primary Language_Unassimilated - Native Language Only',
       'Household Primary Language_nan', 'Religions_Description_Buddhist',
       'Religions_Description_Catholic',
       'Religions_Description_Christian',
       'Religions_Description_Eastern Orthodox',
       'Religions_Description_Greek Orthodox',
       'Religions_Description_Hindu', 'Religions_Description_Islamic',
       'Religions_Description_Jewish', 'Religions_Description_Lutheran',
       'Religions_Description_Protestant', 'Religions_Description_Shinto',
       'Religions_Description_Sikh', 'Religions_Description_nan', 'Presidential_Primary_1992-04-07',
       'Presidential_Primary_1996-03-07',
       'Presidential_Primary_2000-03-07',
       'Presidential_Primary_2004-03-02',
       'Presidential_Primary_2008-02-05',
       'Presidential_Primary_2012-04-24',
       'Presidential_Primary_2016-04-19']].groupby('KEY').sum()

,Broad Ethnic Groupings_East and South Asian,Broad Ethnic Groupings_European,Broad Ethnic Groupings_Hispanic and Portuguese,Broad Ethnic Groupings_Likely African-American,Broad Ethnic Groupings_Other,Broad Ethnic Groupings_nan,Household Primary Language_Assimilated - English Speaking,Household Primary Language_Bilingual - English Primary,Household Primary Language_Bilingual - Native Language Primary,Household Primary Language_Unassimilated - Native Language Only,...,Religions_Description_Shinto,Religions_Description_Sikh,Religions_Description_nan,Presidential_Primary_1992-04-07,Presidential_Primary_1996-03-07,Presidential_Primary_2000-03-07,Presidential_Primary_2004-03-02,Presidential_Primary_2008-02-05,Presidential_Primary_2012-04-24,Presidential_Primary_2016-04-19
KEY,,,,,,,,,,,,,,,,,,,,,
30019,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,1
30038,4,57,14,1,21,14,8,9,0,6,...,0,0,61,1,0,13,15,32,0,81
30056,7,61,10,0,11,14,20,4,3,5,...,2,0,40,1,0,10,15,28,0,72
30064,4,18,5,0,10,9,8,4,0,3,...,0,0,23,1,0,7,6,20,0,33
30066,4,68,18,0,5,13,3,0,3,15,...,0,0,50,1,0,6,11,41,0,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76099,3,78,5,1,5,22,9,2,0,0,...,1,0,46,19,1,23,31,55,0,80
76100,6,91,9,0,3,32,6,4,0,0,...,1,0,53,15,0,36,36,79,0,101
76101,4,47,2,0,4,14,5,1,0,0,...,0,0,16,13,0,22,27,45,0,54


In [20]:
df = df.merge(right = temp_df[['KEY', 'Broad Ethnic Groupings_East and South Asian',
       'Broad Ethnic Groupings_European',
       'Broad Ethnic Groupings_Hispanic and Portuguese',
       'Broad Ethnic Groupings_Likely African-American',
       'Broad Ethnic Groupings_Other', 'Broad Ethnic Groupings_nan',
       'Household Primary Language_Assimilated - English Speaking',
       'Household Primary Language_Bilingual - English Primary',
       'Household Primary Language_Bilingual - Native Language Primary',
       'Household Primary Language_Unassimilated - Native Language Only',
       'Household Primary Language_nan', 'Religions_Description_Buddhist',
       'Religions_Description_Catholic',
       'Religions_Description_Christian',
       'Religions_Description_Eastern Orthodox',
       'Religions_Description_Greek Orthodox',
       'Religions_Description_Hindu', 'Religions_Description_Islamic',
       'Religions_Description_Jewish', 'Religions_Description_Lutheran',
       'Religions_Description_Protestant', 'Religions_Description_Shinto',
       'Religions_Description_Sikh', 'Religions_Description_nan', 'Presidential_Primary_1992-04-07',
       'Presidential_Primary_1996-03-07',
       'Presidential_Primary_2000-03-07',
       'Presidential_Primary_2004-03-02',
       'Presidential_Primary_2008-02-05',
       'Presidential_Primary_2012-04-24',
       'Presidential_Primary_2016-04-19']].groupby('KEY').sum(), on = 'KEY', how = 'inner')

In [25]:
df.to_file("oneEDperline.geojson", driver="GeoJSON")